### Relacionando bibliotecas necessárias para o processamento

In [3]:
from datetime import datetime, date, timedelta
import datetime
import os
import pandas as pd
import regex as re
from openpyxl import Workbook, load_workbook
from openpyxl.workbook import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border, Color, Alignment
from openpyxl.formula.translate import Translator
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from datetime import datetime
import openpyxl
import xlwings as xw
import numpy as np
from workadays import workdays as wd
import xlrd

### Lendo diretório onde estão localizados os arquivos de notas fiscais

In [27]:
diretorio = os.getcwd()

diretorio_resultado = diretorio + '/Resultado/Base Clientes Midori.xlsx'
diretorio_notas = diretorio+ '/Notas'
Lista_Arquivos = os.listdir(diretorio_notas)

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

Base_XML = pd.DataFrame()
Base_Full = pd.DataFrame()

linha=0

Base_Full['Chave_NFE'] = ''
Base_Full['Cod_NFE'] = ''
Base_Full['Data_Emissao'] = ''
Base_Full['CPF/CNPJ'] = ''
Base_Full['Nome'] = ''
Base_Full['Bairro'] = ''
Base_Full['Municipio'] = ''
Base_Full['Estado'] = ''
Base_Full['CEP'] = ''
Base_Full['Telefone'] = ''
Base_Full['Total_Pago'] = ''

for Cod_XML in Lista_Arquivos:
    
    if linha <= len(Lista_Arquivos):
        arquivo = diretorio + '/Notas/'+Cod_XML
        #print(arquivo) 
        Base_XML_NFE = pd.read_xml(arquivo,xpath="//doc:ide",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Dest = pd.read_xml(arquivo,xpath="//doc:dest",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_EnderDest = pd.read_xml(arquivo,xpath="//doc:enderDest",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Produto = pd.read_xml(arquivo,xpath="//doc:prod",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Pagamento = pd.read_xml(arquivo,xpath="//doc:detPag",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Chave_NFE = pd.read_xml(arquivo,xpath="//doc:infNFe",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Item = pd.read_xml(arquivo,xpath="//doc:det",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        
        # Verifica existencia de dados de CPF ou CNPJ
        if 'CPF' in Base_XML_Dest.columns :
            
            Base_XML_EnderDest['CPF/CNPJ'] = Base_XML_Dest['CPF']
            Base_XML_Produto['CPF/CNPJ'] = Base_XML_Dest['CPF']
        else:
            Base_XML_EnderDest['CPF/CNPJ'] = Base_XML_Dest['CNPJ']
            Base_XML_Produto['CPF/CNPJ'] = Base_XML_Dest['CNPJ']
        
        # Verifica existencia de dados de telefone
        if 'fone' in Base_XML_EnderDest.columns :
            
            Base_XML_EnderDest['fone'] = Base_XML_EnderDest['fone']
            
        else:
            Base_XML_EnderDest['fone'] = ''
        
        
        data=str(Base_XML_NFE['dhEmi'])
        data_emissao = data[1:15]
        
        Base_Full = Base_Full.reset_index(drop=True)

        Base_Full.loc[linha,'Chave_NFE'] = Base_XML_Chave_NFE.loc[0,'Id']
        Base_Full.loc[linha,'Cod_NFE'] = Base_XML_NFE.loc[0,'nNF']
        Base_Full.loc[linha,'Data_Emissao'] = data_emissao
        Base_Full.loc[linha,'CPF/CNPJ'] = Base_XML_EnderDest.loc[0,'CPF/CNPJ']
        Base_Full.loc[linha,'Nome'] = Base_XML_Dest.loc[0,'xNome']
        Base_Full.loc[linha,'Bairro'] = Base_XML_EnderDest.loc[0,'xBairro']
        Base_Full.loc[linha,'Municipio'] = Base_XML_EnderDest.loc[0,'xMun']
        Base_Full.loc[linha,'Estado'] = Base_XML_EnderDest.loc[0,'UF']
        Base_Full.loc[linha,'CEP'] = Base_XML_EnderDest.loc[0,'CEP']
        Base_Full.loc[linha,'Telefone'] = Base_XML_EnderDest.loc[0,'fone']
        Base_Full.loc[linha,'Total_Pago'] = Base_XML_Pagamento.loc[0,'vPag']
        
        for i in range(len(Base_XML_Produto)):
            Base_Full.loc[linha,'Produto_'+str(i+1)] = Base_XML_Produto.loc[i,'xProd']
            Base_Full.loc[linha,'Qtd_Produto_'+str(i+1)] = Base_XML_Produto.loc[i,'qCom']
        
        linha=linha+1
    
with pd.ExcelWriter(diretorio_resultado) as escrever:
    Base_Full.to_excel(escrever, sheet_name ='Base_Clientes') 
    
Base_Full.head()

,Chave_NFE,Cod_NFE,Data_Emissao,CPF/CNPJ,Nome,Bairro,Municipio,Estado,CEP,Telefone,Total_Pago
0,NFe33230228570443000137550020000000521912640012,52,2023-02-01,3352181136,Geisiane Lins Pacheco,Guara II,Brasilia,DF,71070612,6199152069,149.90
1,NFe33230228570443000137550010000000221758212715,22,2023-02-03,18682602750,Gabriel Sousa da Silva,Inhauma,Rio de Janeiro,RJ,21061400,21975838075,218.52
2,NFe33230128570443000137550010000000191898209742,19,2023-01-31,1076191649,EVANDRO EUSTAQUIO ALVARES DE MACEDO,Centro,Abaete,MG,35620000,37999250778,1229.89
3,NFe33230128570443000137550010000000121376065012,12,2023-01-30,22244723828,Narjara Neves,Vila Augusta,Guarulhos,SP,7023022,11991003802,77.44
4,NFe33230228570443000137550010000000271081790839,27,2023-02-08,8981377642,Aristides Tranquillini Neto,Bela Vista,Sao Paulo,SP,1311000,1173744403,302.31
